In [1]:
""" This notebook computes optimal portfolio weights for NYSE_2 dataset in the case of logarithmic utility.
The results for 30 experiments of the GDSEG algorithm are written to the file log_opt_portf_NYSE_2.txt
"""
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
# Importing NYSE_2 dataset
stocks=pd.read_csv('NYSE_2.csv')
print(stocks.shape)
stocks.head()

(11178, 19)


,ahp,alcoa,amerb,coke,dow,dupont,ford,ge,gm,hp,ibm,inger,jnj,kimbc,merck,mmm,morris,pandg,schlum
0,1.01515,1.02765,1.04183,1.00637,1.00847,1.01983,1.00000,1.00000,1.01026,1.01935,1.00429,1.01357,0.99683,1.05340,1.03148,1.03377,1.01495,1.00775,1.01176
1,1.01493,1.04036,0.98905,1.00475,1.00840,1.00833,1.00157,1.02187,0.99746,1.01266,0.99573,1.00446,1.00318,1.00461,1.00898,1.00251,1.00000,1.00192,1.01938
2,1.00000,0.97629,0.97786,0.98583,0.99722,0.99449,0.98116,0.97860,0.98219,0.98125,0.98571,0.99556,0.95873,0.98165,0.98043,0.95990,0.97218,0.98656,0.97338
3,1.02451,1.00662,1.02642,1.01917,0.99443,1.00693,1.02720,1.00795,0.98705,1.00637,1.01522,1.00000,1.01325,0.98131,1.01089,1.03655,0.99663,1.00778,1.00000
4,1.03100,0.98465,1.00368,1.00313,1.02801,1.00413,1.04361,1.00394,1.00525,1.03165,1.02427,1.01563,1.00654,1.02381,1.01077,0.99496,0.98649,1.01158,1.01563


In [4]:
# r: array for stock returns
N=stocks.shape[0]
d=stocks.shape[1]
r=np.zeros((N,d))
r=stocks.to_numpy()

In [5]:
def GDSEG(U,R,n_attempts=10**4,threshold=1/10**10):
    """ Greedy doubly stochastic exponentiated graient algorithm (GDSEG)
    U: empirical utility, power or logarithmic 
    R: array of stock pirces
    The function returns the optimal oprtfolio weights: w_old, the optimal value of the objective function: U(w_old,R), and the number of iterations: i
    """
    N=R.shape[0]
    d=R.shape[1]
    w_old=np.ones(d)/d
    w_new=np.zeros(d)
    attempt=0
    i=0
    while attempt<=n_attempts:
        i+=1
        k=np.random.randint(0,N)
        eta=np.random.rand()
        a=[w_old[j]*np.exp(eta*R[k,j]/(np.dot(w_old,R[k,:]))**(1-alpha)) for j in range(d)]
        w_new=a/np.sum(a)
        attempt+=1
        if U(w_new,R)>U(w_old,R)+threshold:
            w_old=w_new
            attempt=0
        #if i%10000==0: print(s,i,100*w_old,U(w_old,R),eta)
    return w_old, U(w_old,R), i

In [6]:
def U(nu,r):
    """ Empirical utility, power (0<alpha<=1) or logarithmic (alpha=0) """
    if alpha==0:
        return np.mean(np.log(np.dot(r[0:N,:],nu)))
    else:
        return np.mean(np.dot(r[0:N,:],nu)**alpha)

In [7]:
# Computation of optimal portfolios
n_experiments=30
alpha=0.0
opt_portf=np.zeros((n_experiments,d))
opt_val=np.zeros(n_experiments)
n_iter=np.zeros(n_experiments)
f=open('log_opt_portf_NYSE_2.txt','ab')
for s in range(n_experiments):
    np.random.seed(1+s)
    opt_portf[s,:], opt_val[s], n_iter[s]  = GDSEG(U,r)
    print('GDSEG experiment:',s,'Number of iterations:',n_iter[s])
    np.savetxt(f,opt_portf[s,:].reshape(1,d))
f.close()

GDSEG experiment: 0 Number of iterations: 76609.0
GDSEG experiment: 1 Number of iterations: 49354.0
GDSEG experiment: 2 Number of iterations: 69340.0
GDSEG experiment: 3 Number of iterations: 109417.0
GDSEG experiment: 4 Number of iterations: 48305.0
GDSEG experiment: 5 Number of iterations: 76294.0
GDSEG experiment: 6 Number of iterations: 76579.0
GDSEG experiment: 7 Number of iterations: 73420.0
GDSEG experiment: 8 Number of iterations: 58426.0
GDSEG experiment: 9 Number of iterations: 69659.0
GDSEG experiment: 10 Number of iterations: 88821.0
GDSEG experiment: 11 Number of iterations: 62463.0
GDSEG experiment: 12 Number of iterations: 98641.0
GDSEG experiment: 13 Number of iterations: 57029.0
GDSEG experiment: 14 Number of iterations: 56517.0
GDSEG experiment: 15 Number of iterations: 101258.0
GDSEG experiment: 16 Number of iterations: 76068.0
GDSEG experiment: 17 Number of iterations: 61866.0
GDSEG experiment: 18 Number of iterations: 86395.0
GDSEG experiment: 19 Number of iteratio

In [8]:
print('Average number of iterations',n_iter.mean())

Average number of iterations 73467.1
